Представим, что мы проводили этот эксперимент на пользователях в течение 2 недель. В результаты мы получили сырые данные об эксперименте, которые нам надо обработать и сделать выводы. 

В результате у нас получилось 2 таблички:

1) Ответы нашего сервиса с рекомендациями  (views) – в них мы знаем, какому пользователю что мы порекомендовали и в какую группу его отнесли. И, конечно, знаем момент времени, когда это произошло.
2) Данные о лайках (likes) — в них мы знаем, какой пользователь и какой пост лайкнул, в том числе момент времени, когда это произошло.

In [1]:
import pandas as pd

import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [2]:
views = pd.read_csv("data/views.csv")
likes = pd.read_csv("data/likes.csv")

In [3]:
views

,user_id,exp_group,recommendations,timestamp
0,128381,control,[3644 4529 4704 5294 4808],1654030803
1,146885,test,[1399 1076 797 7015 5942],1654030811
2,50948,test,[2315 3037 1861 6567 4093],1654030825
3,37703,test,[2842 1949 162 1588 6794],1654030826
4,14661,test,[2395 5881 5648 3417 673],1654030829
...,...,...,...,...
193290,158267,test,[1733 6834 4380 1915 1627],1655240340
193291,63527,control,[2454 191 3873 6404 1588],1655240347
193292,52169,test,[1368 1709 1616 798 5305],1655240354
193293,142402,test,[5895 6984 1978 6548 6106],1655240373


In [4]:
likes

,user_id,post_id,timestamp
0,128381,4704,1654030804
1,146885,1399,1654030816
2,50948,2315,1654030828
3,14661,673,1654030831
4,37703,1588,1654030833
...,...,...,...
230171,31851,5964,1655243535
230172,51512,1498,1655243537
230173,34017,5009,1655243573
230174,13267,1787,1655243692


Найдём случаи, когда пользователь попал в несколько групп.

In [5]:
user_exp_groups_count = views.groupby('user_id')['exp_group'].nunique()
duplicates_idx = user_exp_groups_count[user_exp_groups_count > 1].index
duplicates = views[views['user_id'].isin(duplicates_idx)]

duplicates

,user_id,exp_group,recommendations,timestamp
1311,148670,test,[2992 1368 1261 3901 4471],1654039282
6179,142283,control,[1109 101 5288 4941 132],1654069529
29724,148670,test,[5053 1563 7194 633 1392],1654217190
30748,148670,test,[7128 1023 1388 6807 5945],1654223589
39653,55788,test,[3747 6638 5214 2801 5740],1654279384
39787,25623,test,[1529 6456 1549 4870 4651],1654280154
41040,148670,test,[1866 622 4374 3756 5424],1654288069
45925,142283,control,[4181 3410 751 1880 1682],1654318631
46348,142283,test,[6484 611 395 5678 7295],1654321430
82515,55788,control,[4970 2990 4592 6611 5483],1654545938


Теперь уберём данных пользователей из датасета.

In [6]:
not_duplicates_idx = user_exp_groups_count[user_exp_groups_count == 1].index
filtered_df = views[views['user_id'].isin(not_duplicates_idx)]

filtered_df

,user_id,exp_group,recommendations,timestamp
0,128381,control,[3644 4529 4704 5294 4808],1654030803
1,146885,test,[1399 1076 797 7015 5942],1654030811
2,50948,test,[2315 3037 1861 6567 4093],1654030825
3,37703,test,[2842 1949 162 1588 6794],1654030826
4,14661,test,[2395 5881 5648 3417 673],1654030829
...,...,...,...,...
193290,158267,test,[1733 6834 4380 1915 1627],1655240340
193291,63527,control,[2454 191 3873 6404 1588],1655240347
193292,52169,test,[1368 1709 1616 798 5305],1655240354
193293,142402,test,[5895 6984 1978 6548 6106],1655240373


In [7]:
print("Количаство пользователей в контрольной группе:   ", filtered_df[filtered_df['exp_group'] == 'control']['user_id'].nunique())
print("Количаство пользователей в тестовой группе:      ", filtered_df[filtered_df['exp_group'] == 'test']['user_id'].nunique())

Количаство пользователей в контрольной группе:    32350
Количаство пользователей в тестовой группе:       32659


Найдём долю пользователей (из тех, для которых мы делали рекомендации), которые поставили хотя бы 1 лайк.

In [8]:
rec_users_id = filtered_df['user_id'].unique()
rec_users_likes = likes[likes['user_id'].isin(rec_users_id)]['user_id'].unique()

print(f"Доля пользователей, поставивших хотя бы 1 лайк: {len(rec_users_likes) * 100 / len(rec_users_id):.1f}%")

Доля пользователей, поставивших хотя бы 1 лайк: 89.5%


Рассчитаем долю пользователей с хотя бы одним лайком для обеих групп. 

In [9]:
rec_users = filtered_df.groupby('user_id').first().reset_index()

rec_users_test_id = rec_users[rec_users['exp_group'] == 'test']['user_id']
rec_users_control_id = rec_users[rec_users['exp_group'] == 'control']['user_id']

In [10]:
rec_users_test_id_likes = likes[likes['user_id'].isin(rec_users_test_id)]['user_id'].unique()
rec_users_control_id_likes = likes[likes['user_id'].isin(rec_users_control_id)]['user_id'].unique()

rec_users_test_likes = len(rec_users_test_id_likes) * 100 / len(rec_users_test_id)
rec_users_control_likes = len(rec_users_control_id_likes) * 100 / len(rec_users_control_id)

print(f"Доля пользователей с хотя бы 1 лайком в тестовой группе: {rec_users_test_likes:.1f}%")
print(f"Доля пользователей с хотя бы 1 лайком в контрольной группе: {rec_users_control_likes:.1f}%")

Доля пользователей с хотя бы 1 лайком в тестовой группе: 89.8%
Доля пользователей с хотя бы 1 лайком в контрольной группе: 89.1%


Теперь оценим, является ли различие между числом лайков в тестовой и контрольной группая статистически значимым. Сначала необходимо выбрать статистический критерий.

Проверим метрику "доля пользователей с хотя бы одним лайком". Для её проверки можно использовать z-критерий для долей. В качестве нулевой гипотезы H0 возьмём утверждение, что доли лайкнувших пользователей одинаковы. А в качестве альтернативной гипотезы тогда будет утверждение о том, что доли различаются.

Сразу стоит оговорить, что по заданию необходимо было использовать двухсторонний критерий, однако в данном случае лучше выбрать односторонний, а именно в качестве альтернативной гипотезы взять утверждение о том, что в тестовой группе доля лайкнувших пользователей увеличилась.

In [11]:
from statsmodels.stats.proportion import proportions_ztest

n1 = len(rec_users_test_id)
n2 = len(rec_users_control_id)

x1 = len(rec_users_test_id_likes)
x2 = len(rec_users_control_id_likes)

p1 = x1 / n1
p2 = x2 / n2

_, p_value = proportions_ztest([x1, x2], [n1, n2])

print(f"Тестовая группа: {p1:.4f}, Контрольная группа: {p2:.4f}")
print(f"p-value: {p_value:.4f}")

if p_value < 0.05:
    print("Разница статистически значима (отвергаем H0)")
else:
    print("Нет статистически значимой разницы (не отвергаем H0)")

Тестовая группа: 0.8982, Контрольная группа: 0.8913
p-value: 0.0045
Разница статистически значима (отвергаем H0)


Теперь проверим метрику "число лайков на пользователя". В качестве нулевой гипотезы H0 будет утверждение, что среднее число лайков на пользователя совпадает в обеих группах. А в качестве альтернативной гипотезы тогда будет утверждение о том, что среднее число лайков на пользователя различается.

Сразу стоит оговорить, что в данном случае, как и в предыдущей метрике лучше взять односторонний критерий, а именно в качестве альтернативной гипотезы выбрать утверждение о том, что среднее число лайков на пользователя в тестовой группе увеличилось.

В данном случае необходимо проверить, является ли распределение числа лайков на пользователя нормальным, чтобы понять, можно ли использовать t-критерий.

In [12]:
import scipy.stats as stats

test_group_likes = likes[likes['user_id'].isin(rec_users_test_id)].groupby('user_id').size()
control_group_likes = likes[likes['user_id'].isin(rec_users_control_id)].groupby('user_id').size()

stat_test, p_value_test = stats.shapiro(test_group_likes)
stat_control, p_value_control = stats.shapiro(control_group_likes)

print(f"Тестовая группа: p-value = {p_value_test:.4e}")
print(f"Контрольная группа: p-value = {p_value_control:.4e}")

if p_value_test < 0.05:
    print("Распределение числа лайков на пользователя не является нормальным для тестовой группы")
else:
    print("Распределение числа лайков на пользователя является нормальным для тестовой группы")

if p_value_control < 0.05:
    print("Распределение числа лайков на пользователя не является нормальным для контрольной группы")
else:
    print("Распределение числа лайков на пользователя является нормальным для контрольной группы")

Тестовая группа: p-value = 5.0595e-103
Контрольная группа: p-value = 5.1631e-101
Распределение числа лайков на пользователя не является нормальным для тестовой группы
Распределение числа лайков на пользователя не является нормальным для контрольной группы


Так как данные не являются нормальными, использовать t-критерий в данном случае нельзя, поэтому выбираем критерий Манна-Уитни-Уилкоксона.

In [13]:
from scipy.stats import mannwhitneyu

_, p_value = mannwhitneyu(test_group_likes, control_group_likes)

print(f"p-value: {p_value:.4f}")

if p_value < 0.05:
    print("Разница статистически значима (отвергаем H0)")
else:
    print("Нет статистически значимой разницы (не отвергаем H0)")

p-value: 0.0017
Разница статистически значима (отвергаем H0)


Таким образом, обе метрики показали, что улучшение является статистически значимым.

Теперь посчитаем hitrate – долю рекомендаций, которые пользователи лайкнули. Для этого соберём в одной табличке информацию о показанных постах и соответствующих лайках. У нас нет айдишника, по которому  можно сджойнить их, поэтому сначала соединим таблицы по пользователю, далее оставим только те записи, где лайкнутый пост содоржится в рекомендованных и в конце отфильтруем по timestamp так, чтобы лайк был совершён после показа, но не позднее, чем через час.

Преобразуем рекомендации в целочисленное множество, так как изначально данные пришли в строке.

In [14]:
views['recommendations'] = views['recommendations'].apply(lambda x: set(map(int, str(x)[1:-1].split())))
views

,user_id,exp_group,recommendations,timestamp
0,128381,control,"{4704, 4808, 5294, 4529, 3644}",1654030803
1,146885,test,"{7015, 1076, 5942, 1399, 797}",1654030811
2,50948,test,"{1861, 6567, 2315, 3037, 4093}",1654030825
3,37703,test,"{162, 6794, 1588, 2842, 1949}",1654030826
4,14661,test,"{673, 3417, 5648, 5881, 2395}",1654030829
...,...,...,...,...
193290,158267,test,"{1915, 1733, 6834, 1627, 4380}",1655240340
193291,63527,control,"{3873, 6404, 1588, 2454, 191}",1655240347
193292,52169,test,"{1709, 1616, 1368, 5305, 798}",1655240354
193293,142402,test,"{1978, 5895, 6984, 6548, 6106}",1655240373


Соединим таблицы по пользователю и преобразуем timestamp.

In [15]:

merged = views.merge(likes, on='user_id', how='left', suffixes=('_view', '_like'))
merged['post_id'] = merged['post_id'].fillna(0).astype(int)
merged['timestamp_view'] = pd.to_datetime(merged['timestamp_view'], unit='s')
merged['timestamp_like'] = pd.to_datetime(merged['timestamp_like'], unit='s')
merged

,user_id,exp_group,recommendations,timestamp_view,post_id,timestamp_like
0,128381,control,"{4704, 4808, 5294, 4529, 3644}",2022-05-31 21:00:03,4704,2022-05-31 21:00:04
1,128381,control,"{4704, 4808, 5294, 4529, 3644}",2022-05-31 21:00:03,5294,2022-05-31 21:00:38
2,128381,control,"{4704, 4808, 5294, 4529, 3644}",2022-05-31 21:00:03,3608,2022-06-12 15:55:27
3,128381,control,"{4704, 4808, 5294, 4529, 3644}",2022-05-31 21:00:03,2542,2022-06-12 15:55:42
4,128381,control,"{4704, 4808, 5294, 4529, 3644}",2022-05-31 21:00:03,4165,2022-06-12 16:53:26
...,...,...,...,...,...,...
1017168,52169,test,"{1709, 1616, 1368, 5305, 798}",2022-06-14 20:59:14,1709,2022-06-14 20:59:29
1017169,142402,test,"{1978, 5895, 6984, 6548, 6106}",2022-06-14 20:59:33,6548,2022-06-14 20:59:34
1017170,72259,control,"{3587, 3811, 6117, 6567, 1255}",2022-06-14 20:59:48,1712,2022-06-04 01:11:51
1017171,72259,control,"{3587, 3811, 6117, 6567, 1255}",2022-06-14 20:59:48,5070,2022-06-04 02:09:50


Оставим только те записи, в которых лайкнутый пост находится в рекомендациях.

In [16]:
merged['contains'] = merged.apply(lambda row: row['post_id'] in row['recommendations'], axis=1)
merged

,user_id,exp_group,recommendations,timestamp_view,post_id,timestamp_like,contains
0,128381,control,"{4704, 4808, 5294, 4529, 3644}",2022-05-31 21:00:03,4704,2022-05-31 21:00:04,True
1,128381,control,"{4704, 4808, 5294, 4529, 3644}",2022-05-31 21:00:03,5294,2022-05-31 21:00:38,True
2,128381,control,"{4704, 4808, 5294, 4529, 3644}",2022-05-31 21:00:03,3608,2022-06-12 15:55:27,False
3,128381,control,"{4704, 4808, 5294, 4529, 3644}",2022-05-31 21:00:03,2542,2022-06-12 15:55:42,False
4,128381,control,"{4704, 4808, 5294, 4529, 3644}",2022-05-31 21:00:03,4165,2022-06-12 16:53:26,False
...,...,...,...,...,...,...,...
1017168,52169,test,"{1709, 1616, 1368, 5305, 798}",2022-06-14 20:59:14,1709,2022-06-14 20:59:29,True
1017169,142402,test,"{1978, 5895, 6984, 6548, 6106}",2022-06-14 20:59:33,6548,2022-06-14 20:59:34,True
1017170,72259,control,"{3587, 3811, 6117, 6567, 1255}",2022-06-14 20:59:48,1712,2022-06-04 01:11:51,False
1017171,72259,control,"{3587, 3811, 6117, 6567, 1255}",2022-06-14 20:59:48,5070,2022-06-04 02:09:50,False


In [17]:
filtered = merged[merged['contains']]
filtered

,user_id,exp_group,recommendations,timestamp_view,post_id,timestamp_like,contains
0,128381,control,"{4704, 4808, 5294, 4529, 3644}",2022-05-31 21:00:03,4704,2022-05-31 21:00:04,True
1,128381,control,"{4704, 4808, 5294, 4529, 3644}",2022-05-31 21:00:03,5294,2022-05-31 21:00:38,True
7,146885,test,"{7015, 1076, 5942, 1399, 797}",2022-05-31 21:00:11,1399,2022-05-31 21:00:16,True
11,50948,test,"{1861, 6567, 2315, 3037, 4093}",2022-05-31 21:00:25,2315,2022-05-31 21:00:28,True
16,37703,test,"{162, 6794, 1588, 2842, 1949}",2022-05-31 21:00:26,1588,2022-05-31 21:00:33,True
...,...,...,...,...,...,...,...
1017160,119630,test,"{7077, 3143, 1577, 4588, 599}",2022-06-14 20:58:57,599,2022-06-14 20:59:27,True
1017163,158267,test,"{1915, 1733, 6834, 1627, 4380}",2022-06-14 20:59:00,6834,2022-06-14 20:59:01,True
1017167,63527,control,"{3873, 6404, 1588, 2454, 191}",2022-06-14 20:59:07,3873,2022-06-14 20:59:18,True
1017168,52169,test,"{1709, 1616, 1368, 5305, 798}",2022-06-14 20:59:14,1709,2022-06-14 20:59:29,True


Оставим только те записи, где пост лайкнули после получения рекомендаций, но не позже, чем через час после этого.

In [18]:
filtered = filtered[
    (filtered['timestamp_like'] >= filtered['timestamp_view']) &
    (filtered['timestamp_like'] <= (filtered['timestamp_view'] + pd.Timedelta(hours=1)))
]

filtered

,user_id,exp_group,recommendations,timestamp_view,post_id,timestamp_like,contains
0,128381,control,"{4704, 4808, 5294, 4529, 3644}",2022-05-31 21:00:03,4704,2022-05-31 21:00:04,True
1,128381,control,"{4704, 4808, 5294, 4529, 3644}",2022-05-31 21:00:03,5294,2022-05-31 21:00:38,True
7,146885,test,"{7015, 1076, 5942, 1399, 797}",2022-05-31 21:00:11,1399,2022-05-31 21:00:16,True
11,50948,test,"{1861, 6567, 2315, 3037, 4093}",2022-05-31 21:00:25,2315,2022-05-31 21:00:28,True
16,37703,test,"{162, 6794, 1588, 2842, 1949}",2022-05-31 21:00:26,1588,2022-05-31 21:00:33,True
...,...,...,...,...,...,...,...
1017160,119630,test,"{7077, 3143, 1577, 4588, 599}",2022-06-14 20:58:57,599,2022-06-14 20:59:27,True
1017163,158267,test,"{1915, 1733, 6834, 1627, 4380}",2022-06-14 20:59:00,6834,2022-06-14 20:59:01,True
1017167,63527,control,"{3873, 6404, 1588, 2454, 191}",2022-06-14 20:59:07,3873,2022-06-14 20:59:18,True
1017168,52169,test,"{1709, 1616, 1368, 5305, 798}",2022-06-14 20:59:14,1709,2022-06-14 20:59:29,True


Так как нам важен только факт наличия лайка, сгруппируем записи по пользователю и времени рекомендации.

In [19]:
hitrate_data = filtered.groupby(['user_id', 'timestamp_view']).size().reset_index(name='like_count')
hitrate_data

,user_id,timestamp_view,like_count
0,200,2022-06-12 04:44:07,1
1,201,2022-06-05 15:58:24,1
2,201,2022-06-08 04:53:59,2
3,202,2022-06-06 01:55:54,2
4,212,2022-06-01 16:59:15,3
...,...,...,...
137866,168541,2022-06-12 06:43:57,2
137867,168545,2022-06-01 02:34:05,2
137868,168545,2022-06-03 22:28:01,1
137869,168545,2022-06-05 06:56:24,1


In [20]:
views = views.rename(columns={'timestamp': 'timestamp_view'})
views['timestamp_view'] = pd.to_datetime(views['timestamp_view'], unit='s')
views

,user_id,exp_group,recommendations,timestamp_view
0,128381,control,"{4704, 4808, 5294, 4529, 3644}",2022-05-31 21:00:03
1,146885,test,"{7015, 1076, 5942, 1399, 797}",2022-05-31 21:00:11
2,50948,test,"{1861, 6567, 2315, 3037, 4093}",2022-05-31 21:00:25
3,37703,test,"{162, 6794, 1588, 2842, 1949}",2022-05-31 21:00:26
4,14661,test,"{673, 3417, 5648, 5881, 2395}",2022-05-31 21:00:29
...,...,...,...,...
193290,158267,test,"{1915, 1733, 6834, 1627, 4380}",2022-06-14 20:59:00
193291,63527,control,"{3873, 6404, 1588, 2454, 191}",2022-06-14 20:59:07
193292,52169,test,"{1709, 1616, 1368, 5305, 798}",2022-06-14 20:59:14
193293,142402,test,"{1978, 5895, 6984, 6548, 6106}",2022-06-14 20:59:33


Соеденим таблицу показов с таблицей поставленных лайков по пользователю и времени рекомендации.

In [21]:
final_data = views.merge(hitrate_data, on=['user_id', 'timestamp_view'], how='left')
final_data['like_count'] = final_data['like_count'].fillna(0)
final_data['like_count'] = final_data['like_count'].astype(int)
final_data

,user_id,exp_group,recommendations,timestamp_view,like_count
0,128381,control,"{4704, 4808, 5294, 4529, 3644}",2022-05-31 21:00:03,2
1,146885,test,"{7015, 1076, 5942, 1399, 797}",2022-05-31 21:00:11,1
2,50948,test,"{1861, 6567, 2315, 3037, 4093}",2022-05-31 21:00:25,1
3,37703,test,"{162, 6794, 1588, 2842, 1949}",2022-05-31 21:00:26,2
4,14661,test,"{673, 3417, 5648, 5881, 2395}",2022-05-31 21:00:29,1
...,...,...,...,...,...
193290,158267,test,"{1915, 1733, 6834, 1627, 4380}",2022-06-14 20:59:00,1
193291,63527,control,"{3873, 6404, 1588, 2454, 191}",2022-06-14 20:59:07,1
193292,52169,test,"{1709, 1616, 1368, 5305, 798}",2022-06-14 20:59:14,1
193293,142402,test,"{1978, 5895, 6984, 6548, 6106}",2022-06-14 20:59:33,1


In [22]:
hitrate = (final_data['like_count'] > 0).mean()

print(f"Hitrate: {hitrate:.4f}")

Hitrate: 0.7133


Теперь применим бакетный подход и t-тест для оценки различия между группами. Сравним результаты до и после бакетирования.

In [23]:
control_hitrate = (final_data[final_data['exp_group'] == 'control']['like_count'] > 0).mean()
test_hitrate = (final_data[final_data['exp_group'] == 'test']['like_count'] > 0).mean()

hitrate_difference = test_hitrate - control_hitrate
hitrate_difference_pct = hitrate_difference * 100

control_group = final_data[final_data['exp_group'] == 'control']['like_count']
test_group = final_data[final_data['exp_group'] == 'test']['like_count']

t_stat, p_value = stats.ttest_ind(control_group, test_group)

if p_value < 0.05:
    if hitrate_difference_pct > 0:
        print(f"В тестовой группе hitrate выше на {int(hitrate_difference_pct)} п.п. (до бакетирования), есть статистически значимая разница.")
    else:
        print(f"В тестовой группе hitrate ниже на {abs(int(hitrate_difference_pct))} п.п. (до бакетирования), есть статистически значимая разница.")
else:
    print(f"Нет статистически значимой разницы между hitrate в группах.")

В тестовой группе hitrate выше на 1 п.п. (до бакетирования), есть статистически значимая разница.


In [24]:
import numpy as np
from scipy.stats import ttest_ind

final_data['bucket'] = np.random.randint(0, 100, size=len(final_data))

final_data['clicked'] = final_data['like_count'] > 0

control_data = final_data[final_data['exp_group'] == 'control']
test_data = final_data[final_data['exp_group'] == 'test']

control_hitrate_by_bucket = control_data.groupby('bucket')['clicked'].mean()
test_hitrate_by_bucket = test_data.groupby('bucket')['clicked'].mean()

t_stat, p_value = ttest_ind(control_hitrate_by_bucket, test_hitrate_by_bucket)

hitrate_difference = test_hitrate_by_bucket.mean() - control_hitrate_by_bucket.mean()
hitrate_difference_pct = hitrate_difference * 100

if p_value < 0.05:
    if hitrate_difference_pct > 0:
        print(f"В тестовой группе hitrate выше на {int(hitrate_difference_pct)} п.п. (после бакетирования), есть статистически значимая разница.")
    else:
        print(f"В тестовой группе hitrate ниже на {abs(int(hitrate_difference_pct))} п.п. (после бакетирования), есть статистически значимая разница.")
else:
    print(f"Нет статистически значимой разницы между hitrate в группах.")

В тестовой группе hitrate выше на 1 п.п. (после бакетирования), есть статистически значимая разница.


Таким образом, все рассмотренные метрики показали, что улучшение является статистически значимым.